#Uploading TMX extraction of al tickers

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Read the dataset from Google Drive, specifying the encoding as 'latin1' or 'ISO-8859-1'
dataset = pd.read_csv('/content/drive/MyDrive/###Your File Path###/tsx.csv', encoding='latin1')

# Drop the column named 'blank' from the dataset
dataset.drop(columns=['blank'], inplace=True)

# Drop rows with any missing values (NaN) and reset the index
dataset = dataset.dropna().reset_index(drop=True)

# Rename columns in the dataset
dataset.rename(columns={'blank 2': 'Ticker TMX', 'blank href': 'TMX Link', 'tablescraper-selected-row': 'Company'}, inplace=True)


In [8]:
dataset

,tablescraper-selected-row,blank href,blank 2
0,A&W Revenue Royalties Income Fund,https://money-tmx-com/en/quote/AW-UN/company,AW-UN
1,Aberdeen International Inc-,https://money-tmx-com/en/quote/AAB/company,AAB
2,abrdn Asia-Pacific Income Fund VCC,https://money-tmx-com/en/quote/FAP/company,FAP
3,Acadian Timber Corp-,https://money-tmx-com/en/quote/ADN/company,ADN
4,Accelerate Absolute Return Hedge Fund,https://money-tmx-com/en/quote/HDGE/company,HDGE
...,...,...,...
2397,First Trust Value Line Dividend Index ETF (CAD...,https://money-tmx-com/en/quote/FUD/company,FUD
2398,First Trust Vest U-S- Equity Buffer ETF - August,https://money-tmx-com/en/quote/AUGB-F/company,AUGB-F
2399,First Trust Vest U-S- Equity Buffer ETF - Febr...,https://money-tmx-com/en/quote/FEBB-F/company,FEBB-F
2400,First Trust Vest U-S- Equity Buffer ETF - May,https://money-tmx-com/en/quote/MAYB-F/company,MAYB-F


In [46]:
# Refine to get just companies from the dataset by removing all Funds, ETFs, Trusts, and REITs.
filtered_df = dataset[~dataset['Company'].str.contains('Fund|ETF|Trust|REIT')]

# Reset the index of the filtered dataset
filtered_df = filtered_df.reset_index(drop=True)
filtered_df

,Company,TMX Link,Ticker TMX,Ticker
0,Aberdeen International Inc-,https://money-tmx-com/en/quote/AAB/company,AAB,AAB.TO
1,Acadian Timber Corp-,https://money-tmx-com/en/quote/ADN/company,ADN,ADN.TO
2,Accord Financial Corp-,https://money-tmx-com/en/quote/ACD/company,ACD,ACD.TO
3,Aclara Resources Inc-,https://money-tmx-com/en/quote/ARA/company,ARA,ARA.TO
4,Adcore Inc-,https://money-tmx-com/en/quote/ADCO/company,ADCO,ADCO.TO
...,...,...,...,...
815,Yellow Pages Limited,https://money-tmx-com/en/quote/Y/company,Y,Y.TO
816,Yorbeau Resources Inc-,https://money-tmx-com/en/quote/YRB/company,YRB,YRB.TO
817,Sleep Country Canada Holdings Inc-,https://money-tmx-com/en/quote/ZZZ/company,ZZZ,ZZZ.TO
818,5N Plus Inc-,https://money-tmx-com/en/quote/VNP/company,VNP,VNP.TO


#Cleaning Data

 First we get the ticker value from Yahoo to be used with yfinance. Doing so by searching for the company on Google with the ca.finance.yahoo.com type and then cleaning that returned dataset. The Google search and extraction is a lengthy process and therefore, the steps are **already done and the returning dataset is provided as the TSX_Companies.csv file**.


In [16]:
def extract_ticker_from_url(url):
    # First try to extract ticker using the original method Also make it capitalized.
    match = re.search(r"https://ca.finance.yahoo.com/quote/([^/.]+\.to)/?", url, re.IGNORECASE)
    if match:
        return match.group(1).str.upper()

    # If the original method fails, try the new method Also make it capitalized.
    match = re.search(r'quote/([^/]+)/?', url)
    if match:
        return match.group(1).str.upper()

    # If both methods fail, return None
    return None

In [ ]:
# DOES NOT NEED TO BE RUN ON THE UPDATED DATA SET -> TSX_Companies.csv
## Was used to change tmx encoding to Yahooo encoding for tickers.
from googlesearch import search
import re

# Iterate through each row in the dataset
for index, row in filtered_df.iterrows():
    try:
        print(f"Processing row {index + 1}...")
        # Define the query for Google search
        query = f"{row['Company']} site:ca.finance.yahoo.com/quote"

        # Perform a Google search and get the URLs
        search_results = list(search(query, num=1, stop=1, pause=1))
        print(f"Search results for '{query}': {search_results}")

        # Extract the ticker from the first link
        if search_results:
            yahoo_finance_url = search_results[0]
            extracted_ticker = extract_ticker_from_url(yahoo_finance_url)
            print(f"extracted_ticker: {extracted_ticker}\n")
            # Update the 'Ticker' column in the DataFrame
            filtered_df.at[index, 'Ticker'] = extracted_ticker
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Print the updated dataset
print(filtered_df)

In [23]:
# Create a new DataFrame without rows where Ticker is None
new_df = filtered_df[filtered_df['Ticker'].isna()]

In [24]:
new_df

,Company,TMX Link,Ticker TMX,Ticker
50,Arrow Long/Short Alternative Class,https://money-tmx-com/en/quote/ACAA/company,ACAA,None


In [ ]:
# Using new_df as the DataFrame containing tickers. There are some american tickers in this dataset that need to be removed.
new_df['Ticker'] = "ACAA.TO"

In [26]:
merged_df = filtered_df

# Assuming 'Name' column is common between the two dataframes
name_to_ticker_mapping = new_df.set_index('Company')['Ticker'].to_dict()

# Map the Ticker values from new_df_CAD to filtered_df based on the 'Name' column
merged_df['Ticker'] = filtered_df['Company'].map(name_to_ticker_mapping).fillna(filtered_df['Ticker'])

In [32]:
merged_df

,Company,TMX Link,Ticker TMX,Ticker
0,Aberdeen International Inc-,https://money-tmx-com/en/quote/AAB/company,AAB,AAB.TO
1,Acadian Timber Corp-,https://money-tmx-com/en/quote/ADN/company,ADN,ADN.TO
2,Accord Financial Corp-,https://money-tmx-com/en/quote/ACD/company,ACD,ACD.TO
3,Aclara Resources Inc-,https://money-tmx-com/en/quote/ARA/company,ARA,ARA.TO
4,Adcore Inc-,https://money-tmx-com/en/quote/ADCO/company,ADCO,ADCO.TO
...,...,...,...,...
815,Yellow Pages Limited,https://money-tmx-com/en/quote/Y/company,Y,Y.TO
816,Yorbeau Resources Inc-,https://money-tmx-com/en/quote/YRB/company,YRB,YRB.TO
817,Sleep Country Canada Holdings Inc-,https://money-tmx-com/en/quote/ZZZ/company,ZZZ,ZZZ.TO
818,5N Plus Inc-,https://money-tmx-com/en/quote/VNP/company,VNP,VNP.TO


In [ ]:
# Save the shrunk dataset to a new CSV file
merged_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/TSX_Companies.csv', index=False, sep=',')

# Provide a download link for the file
from google.colab import files
files.download('/content/drive/MyDrive/Colab Notebooks/data/TSX_Companies.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Getting yFiance Data

In [ ]:
import pandas as pd
import yfinance as yf

#DATABASE OF ALL TICKERS AND SOME ATTRIBUTES.

# Create an empty DataFrame to store the results
TSX_Data = pd.DataFrame(columns=["Ticker", "Company", "Sector", "Revenue_2022", "Revenue_2021", "Revenue_2020"])

# Iterate through each row in the dataset
for index, row in merged_df.iterrows():
    try:
        print(f"Processing row {index + 1}...")
        # Fetch the company information
        ticker = row['Ticker']
        company = yf.Ticker(ticker)

        company_name = company.info.get('longName')
        print(f"Company: {company_name} Ticker: {ticker}")

        # Get the sector information
        sector = company.get_info()['sector'] if 'sector' in company.get_info() else company.get_info()['legalType']
        print(f"sector: {sector}\n")
        # Get the revenue data
        revenue = None

        if not company.incomestmt.empty and 'Total Revenue' in company.incomestmt.index:
            revenue = company.incomestmt.loc['Total Revenue']
        else:
            revenue = None

        if revenue is not None:
            keys = list(revenue.keys())
            revenue_2022 = revenue.get(keys[0], None) if len(keys) > 0 else None
            revenue_2021 = revenue.get(keys[1], None) if len(keys) > 1 else None
            revenue_2020 = revenue.get(keys[2], None) if len(keys) > 2 else None
        else:
            revenue_2022 = revenue_2021 = revenue_2020 = "N/A"

        # Append the symbol, sector, and EBITDA to the DataFrame
        TSX_Data.loc[len(TSX_Data)] = [ticker, company_name, sector, revenue_2022, revenue_2021, revenue_2020, ]
    except Exception as e:
            # If an error occurs (e.g., ticker not found), append None for the sector and revenue
            TSX_Data.loc[len(TSX_Data)] = [ticker, company_name, None, None, None, None]
            print(f"Error processing row {index + 1}: {e}")


In [38]:
# Create a new DataFrame without rows where Ticker is None
TSX_Data = TSX_Data.dropna().drop_duplicates(subset=['Ticker']).reset_index(drop=True)
TSX_Data

,Ticker,Company,Sector,Revenue_2022,Revenue_2021,Revenue_2020
0,AAB.TO,Aberdeen International Inc.,Financial Services,-8762676.0,9139536.0,5628492.0
1,ADN.TO,Acadian Timber Corp.,Basic Materials,90473000.0,95729000.0,91031000.0
2,ACD.TO,Accord Financial Corp.,Financial Services,43403627.0,47593755.0,33904916.0
3,ARA.TO,Aclara Resources Inc.,Basic Materials,0.0,0.0,0.0
4,ADCO.TO,Adcore Inc.,Technology,19693000.0,27966000.0,16997000.0
...,...,...,...,...,...,...
708,XAM.TO,Xanadu Mines Limited,Basic Materials,N/A,N/A,N/A
709,XTG.TO,Xtra-Gold Resources Corp.,Basic Materials,0.0,0.0,0.0
710,XTRA.TO,Xtract One Technologies Inc.,Technology,4111244.0,3619214.0,1081975.0
711,Y.TO,Yellow Pages Limited,Communication Services,268278000.0,287646000.0,333538000.0


In [44]:
# Save the shrunk dataset to a new CSV file
TSX_Data.to_csv('/content/drive/MyDrive/Colab Notebooks/data/TSX_Companies_yf.csv', index=False, sep=',')

# Provide a download link for the file
from google.colab import files
files.download('/content/drive/MyDrive/Colab Notebooks/data/TSX_Companies_yf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Graphing the industries

In [43]:
import pandas as pd
import plotly.express as px
import numpy as np

# Convert revenue columns to numeric format, replacing non-numeric values with NaN
TSX_Data['Revenue_2020'] = pd.to_numeric(TSX_Data['Revenue_2020'], errors='coerce')
TSX_Data['Revenue_2021'] = pd.to_numeric(TSX_Data['Revenue_2021'], errors='coerce')
TSX_Data['Revenue_2022'] = pd.to_numeric(TSX_Data['Revenue_2022'], errors='coerce')

# Divide revenue by 1,000,000 to convert to millions
TSX_Data['Revenue_2020'] /= 1_000_000
TSX_Data['Revenue_2021'] /= 1_000_000
TSX_Data['Revenue_2022'] /= 1_000_000

# Group the data by 'Sector' and calculate total revenue for each sector
sector_revenue = TSX_Data.groupby('Sector')[['Revenue_2022']].sum()

# Reset index to make 'Sector' a column again
sector_revenue = sector_revenue.reset_index()

# Create a bar plot using Plotly
fig = px.bar(sector_revenue, x='Sector', y='Revenue_2022',
             title='Total Revenue in 2022 by Sector (in Millions)',
             labels={'Revenue_2022': 'Revenue (Millions)', 'Sector': 'Sector'},
             color='Sector')

# Show the plot
fig.show()
